<a href="https://colab.research.google.com/github/Shlok-Agarwal-7/DL-Assignment-2/blob/main/DL_assignment_2_Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install transformers datasets tqdm
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
# Import necessary packages
import os
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

In [3]:
os.environ["WANDB_DISABLED"] = "true"

In [4]:
import kagglehub
download_path = kagglehub.dataset_download("paultimothymooney/poetry")
print("Dataset downloaded to:", download_path)

Dataset downloaded to: /kaggle/input/poetry


In [5]:
# Read and collect lyrics
all_lyrics = []
for filename in os.listdir(download_path):
    path = os.path.join(download_path, filename)
    with open(path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read().strip()
        if content:
            all_lyrics.append(content)

print(f"Loaded {len(all_lyrics)} song lyric files.")

Loaded 49 song lyric files.


In [6]:
# Prepare dataset
lyrics_dataset = Dataset.from_dict({"text": all_lyrics})
lyrics_split = lyrics_dataset.train_test_split(test_size=0.1)

In [7]:
# Load tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 has no pad token by default

model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# Tokenize function
def prepare_data(batch):
    encoded = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=512)
    encoded["labels"] = encoded["input_ids"].copy()
    return encoded

tokenized_data = lyrics_split.map(prepare_data, batched=True, remove_columns=["text"])

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [9]:
# Training configuration
config = TrainingArguments(
    output_dir="./output_gpt2_lyrics",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none"  # explicitly disable wandb/other tracking
)

# Setup trainer
trainer = Trainer(
    model=model,
    args=config,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"]
)

In [10]:
# Start fine-tuning
trainer.train()

# Save fine-tuned model and tokenizer
model.save_pretrained("./lyrics-gpt2")
tokenizer.save_pretrained("./lyrics-gpt2")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


('./lyrics-gpt2/tokenizer_config.json',
 './lyrics-gpt2/special_tokens_map.json',
 './lyrics-gpt2/vocab.json',
 './lyrics-gpt2/merges.txt',
 './lyrics-gpt2/added_tokens.json')

In [11]:
seed_line = "Eighteen years eighteen years"
input_tokens = tokenizer(seed_line, return_tensors="pt").to(model.device)

generated_output = model.generate(
    **input_tokens,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    do_sample=True,
    early_stopping=True
)

print("\n Generated Lyrics \n")
print(tokenizer.decode(generated_output[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:679: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



 Generated Lyrics 

Eighteen years eighteen years, thirty-five years in the US. He's not a big guy, but he's a long shot.

"I just didn't really like to go to the same thing as the people in Paris that are the worst. I was just like the other guy who was at work, and I tried to make the best friends in all of the time. So I went to a movie theater in Mexico with my friends and they said, "It's OK for
